## osumapper #4: New Map Reader

1.   List item
2.   List item




In [1]:
#@title Default title text
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Joe's cell
DRIVE_PATH = '/content/drive/MyDrive/Cal/Colab/DeepMusicVisualizer'

In [ ]:
# Richard's cell
DRIVE_PATH = '/content/drive/MyDrive/cs194/DeeperMusicVisualizer'

In [3]:
%cd $DRIVE_PATH

/content/drive/MyDrive/Cal/Colab/DeepMusicVisualizer


In [4]:
%cd rhythm

/content/drive/MyDrive/Cal/Colab/DeepMusicVisualizer/rhythm


In [5]:
%load_ext autoreload
%autoreload 2

Set the input file string to a timed (having the right BPM/offset) .osu file.

It converts the map/music to Python readable format.

In [6]:
!pip install essentia

     |████████████████████████████████| 12.0MB 14.5MB/s 


In [7]:
%reload_ext autoreload
from act_newmap_prep import *

# input file here! (don't remove the "r" before string)
#file_path = r'../data/beatmaps_57/Faded/beatmap.osu'

# Or use auto timing with music file only!!
from act_timing import *;
music_path = r"../data/beatmaps_57/Faded/audio.mp3"

file_path = get_timed_osu_file(music_path, game_mode=0);
print(file_path)
step4_read_new_map(file_path);

BPM: 89.99308013916016
Offset: 708.2086205482483
Final BPM: 90.0
timing.osu


In [ ]:
# file_path = 'timing.osu'
# file_path = 'timing_hardcoded_singletiming.osu'
file_path = 'timing_hardcoded_alltiming.osu'
step4_read_new_map(file_path);

## osumapper #5: Rhythm Predictor

Calculates a map's rhythm based on the music and timing.

Parameters:

"note_density" determines how many notes will be placed on the timeline, ranges from 0 to 1.<br>
"slider_favor" determines how the model favors sliders against circles, ranges from -1 to 1.<br>
"dist_multiplier" determines the distance snap. ranges from 0 to +∞. Of course 0/+∞ are not advisable.<br>
"divisor_favor" determines how the model favors notes to be on X divisors starting from a beat (white, blue, red, blue), ranges from -1 to 1 each.<br>
"slider_max_ticks" determines the max amount of time a slider can slide, ranges from 1 to +∞.

In [ ]:
%reload_ext autoreload
from act_rhythm_calc import *

model = step5_load_model("rhythm_model_8_LSTM128_overfit.h5");
npz = step5_load_npz();
params = step5_set_params(dist_multiplier=1, note_density=0.35, slider_favor=0, divisor_favor=[0] * 8, slider_max_ticks=8);

predictions = step5_predict_notes(model, npz, params);
converted = step5_convert_sliders(predictions, params);

(113, 16, 5)
s (1808, 5) [-1. -1. -1. -1. -1.]
(1808, 11) (113, 16, 11) 8
[-0.9907979]
(1808, 1)
others (1808, 1) [0.]
another (1808, 5)
anoter [0. 0. 0. 0. 0.]
632 notes predicted.
(1808, 1) (1808, 5) (1821,) (1821,) (1808, 11)
[130 297 463 630 797]
[0 1 2 3 4]
384
(393,) (393, 5) (393,) (393,) (393,) (393,) (393,) (393,) (393,)
slider preds [1. 0. 1. 0. 0.]
slider ticks [4 4 8 0 2]
timestamps [ 130 1463 2797 4797 5130]
ticks [ 0  8 16 28 30]
1 16 2797 1.0 0.0 0.0


In [ ]:
# this code is buggy so i copied the content of the method and modified it below (added .npz to end of save)
# step5_save_predictions(converted);

objs, predictions, ticks, timestamps, is_slider, is_spinner, is_note_end, sv, slider_ticks, dist_multiplier = converted;

np.savez_compressed("rhythm_data_faded_128_50epochs_hardcoded_alltiming.npz",
    objs = objs,
    predictions = predictions,
    ticks = ticks,
    timestamps = timestamps,
    is_slider = is_slider,
    is_spinner = is_spinner,
    is_note_end = is_note_end,
    sv = sv,
    slider_ticks = slider_ticks,
    dist_multiplier = dist_multiplier);

# Script to generate model outputs on multiple songs

In [14]:
from act_newmap_prep import *
from act_timing import *;
from act_rhythm_calc import *
import numpy as np
import os, sys

model = step5_load_model("rhythm_model_8_LSTM128_overfit.h5");
params = step5_set_params(dist_multiplier=1, note_density=0.35, slider_favor=0, divisor_favor=[0] * 8, slider_max_ticks=8);

songlist = ["Gorgeous", "PaperThin", "HeatWaves", "IWannaKnow"]

for song in songlist:
  music_path = os.path.join(DRIVE_PATH, f"data/songs/{song}/audio.mp3")
  file_path = get_timed_osu_file(music_path, game_mode=0);
  step4_read_new_map(file_path);
  npz = step5_load_npz();

  predictions = step5_predict_notes(model, npz, params);
  converted = step5_convert_sliders(predictions, params);

  objs, predictions, ticks, timestamps, is_slider, is_spinner, is_note_end, sv, slider_ticks, dist_multiplier = converted;

  result = []
  sliding = False;
  for i, pred in enumerate(predictions):
    if pred[2]==1: #slider
      slide_ticks = slider_ticks[i]
      if i == len(predictions) - 1:
        slide_ms = 1000
      elif ticks[i+1] - ticks[i] > 0:
        ms_gap = timestamps[i+1] - timestamps[i]
        slide_ms = slide_ticks / (ticks[i+1] - ticks[i]) * ms_gap
      else:
        ms_gap = timestamps[i+1] - timestamps[i]
        slide_ms = slide_ticks / (4) * ms_gap
      if sliding==False or i==len(predictions) - 1:
        result.append([timestamps[i], 1, slide_ms])
      else:
        result[-1][2] += slide_ms
      if i!=len(predictions) - 1 and ticks[i+1] - ticks[i] == slide_ticks:
        print('continuing slide')
        sliding = True;
    elif pred[1]==1: #note
      sliding=False;
      result.append([timestamps[i], 0, 0])
    else:
      print("Prediction skipped")

  np.save(os.path.join(DRIVE_PATH, f'data/songs/{song}/output_128_50epochs_essentia.npy'), result)

BPM: 89.81900024414062
Offset: 673.3786463737488
Final BPM: 89.81900024414062
(143, 16, 5)
s (2288, 5) [-1. -1. -1. -1. -1.]
(2288, 11) (143, 16, 11) 8
[-0.98541594]
(2288, 1)
others (2288, 1) [0.]
another (2288, 5)
anoter [0. 0. 0. 0. 0.]
800 notes predicted.
(2288, 1) (2288, 5) (2297,) (2297,) (2288, 11)
[ 673  840 1007 1174 1341]
[0 1 2 3 4]
451
(515,) (515, 5) (515,) (515,) (515,) (515,) (515,) (515,) (515,)
slider preds [1. 0. 1. 0. 0.]
slider ticks [6 2 2 0 0]
timestamps [ 673 2009 2677 3345 3679]
ticks [ 0  8 12 16 18]
1 12 2677 1.0 0.0 0.0
Prediction skipped
Prediction skipped
Prediction skipped
Prediction skipped
Prediction skipped
Prediction skipped
Prediction skipped
Prediction skipped
Prediction skipped
Prediction skipped
Prediction skipped
Prediction skipped
Prediction skipped
Prediction skipped
Prediction skipped
BPM: 103.86103057861328
Offset: 568.8889026641846
Final BPM: 103.86103057861328
(166, 16, 5)
s (2656, 5) [-1. -1. -1. -1. -1.]
(2656, 11) (166, 16, 11) 8
[-0.998